In [6]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py

--2024-06-24 17:35:24--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-24 17:35:24 (25.7 MB/s) - ‘minsearch.py.1’ saved [3832/3832]



In [7]:
import minsearch

In [105]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

--2024-06-24 15:50:01--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642.90K  --.-KB/s    in 0.006s  

2024-06-24 15:50:01 (98.0 MB/s) - ‘documents.json’ saved [658332/658332]



In [8]:
import json

In [23]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [24]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [25]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [26]:
SELECT * WHERE course = 'data-engineering-zoomcamp';

SyntaxError: invalid syntax (3870143388.py, line 1)

In [27]:
index.fit(documents)

In [28]:
import os

os.environ["GROQ_API_KEY"] = "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"

from groq import Groq

client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

In [33]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict ={'course' :'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )
    return results

In [34]:
def build_prompt(query, search_results):
    prompt_template = '''
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE
    
    QUESTION: {question}
    
    CONTEXT: 
    {context}
    '''.strip()
    context = ""
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [35]:
def llm(prompt):
    response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama3-8b-8192",
    )
    return response.choices[0].message.content

In [38]:
query = "how do I run kafka?"

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [39]:
rag(query)

'Based on the context, I can see that you are asking how to run Kafka. I will search the context for an answer to your question.\n\nThe closest answer I can find is related to running Kafka-related programs in the terminal. Here is what I found:\n\n* There is an answer for running a Java producer/consumer/etc in the terminal, which is: `java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java`\n* There is also an answer for running a Python producer: `python producer.py` (but there is also a note about creating a virtual environment and installing packages first)\n\nUnfortunately, I did not find a direct answer on how to run Kafka itself. It is possible that Kafka needs to be run separately or with specific configuration, but the provided answers do not provide the exact information on how to do it.\n\nTherefore, the answer to your question is: NONE'